**I have used RandomForestRegressor to get RMSE of 0.77 which can be further improved to a better value.I have dropped some attributes with null values and used One hot encoding for categorical data.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
airbnb = pd.read_csv("../input/us-airbnb-open-data/AB_US_2020.csv")


In [ ]:
airbnb.head()

In [ ]:
airbnb.info()

In [ ]:
airbnb.isnull().sum()

In [ ]:
import seaborn as sns
sns.barplot(x='room_type',y='price',data=airbnb,palette='spring')


Here we can see that price depends on the type of room and shared room are of lowest price and hotel room are of highest price.

In [ ]:
sns.lmplot(x='number_of_reviews',y='price',data=airbnb, palette='spring')


In [ ]:
airbnb['neighbourhood_group'].fillna('Others',inplace=True)
airbnb.drop(['name','host_name'],axis=1,inplace=True)
airbnb['last_review'] = pd.to_datetime(airbnb['last_review'],infer_datetime_format=True)
airbnb['reviews_per_month'].fillna(airbnb['reviews_per_month'].mean(),inplace=True)
airbnb["last_review"] = airbnb["last_review"].replace(np.nan, airbnb["last_review"].mode().iloc[0])
airbnb.drop(['id','host_id'],axis=1,inplace=True)


Here we dropped 'name' and 'host_name' as price don't depend on them. And change the format of 'last_review'. And replaced the missing values of 'reviews_per_month' with it's mean value. 

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
airbnb['neighbourhood_group'] = label_encoder.fit_transform(airbnb['neighbourhood_group'])
airbnb['neighbourhood'] = label_encoder.fit_transform(airbnb['neighbourhood'])
airbnb['room_type'] = label_encoder.fit_transform(airbnb['room_type'])
airbnb['city'] = label_encoder.fit_transform(airbnb['city'])

Here I have used LabelEncoder for categorical variable. 

In [ ]:
plt.rcParams['figure.figsize']=12,12
g = sns.heatmap(airbnb.corr(),annot=True)



In [ ]:
import datetime 
airbnb['last_review'] = airbnb['last_review'].map(datetime.datetime.toordinal)


In [ ]:
from scipy import stats
z_scores = stats.zscore(airbnb)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_airbnb = airbnb[filtered_entries]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#scaler.fit(new_df)
scaled_features = scaler.fit_transform(new_airbnb)
scaled_features_airbnb = pd.DataFrame(scaled_features, index=new_airbnb.index, columns=new_airbnb.columns)

In [ ]:
scaled_features_airbnb.head()



**SPLITTING OF DATA**

In [ ]:
from sklearn.model_selection import train_test_split
X = scaled_features_airbnb.drop('price',axis=1)
y = scaled_features_airbnb['price']
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.2, random_state=42)


**RANDOM FOREST REGRESSOR**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
model=regressor.fit(X_train, y_train)  
y1 = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
import math
print('MSE',mean_squared_error(y_test, y1))
print('RMSE',math.sqrt(mean_squared_error(y_test, y1)))
print('Adj R^2 value:',1 - (1-regressor.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))